<a href="https://colab.research.google.com/github/IA2021UR/practica-9-joferne/blob/main/practica9_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 9 Parte 3: Desarrollando un modelo de lenguaje para generar texto

Un modelo de lenguaje puede predecir la siguiente palabra de una secuencia basándose en palabras observadas anteriormente. Las redes neuronales son el método más utilizado para desarrollar este tipo de modelos porque pueden usar una representación donde palabras con significados similares tienen representaciones similares. 

En esta parte de la práctica vamos a ver cómo generar uno de esos modelos. 

Este notebook está basado en el libro Deep Learning for Natural Language Processing de Jason Brownlee. 

Es importante que tengas activado el uso de **GPU** en el notebook de colab (menú Edit -> Notebook Settings -> Hardware accelerator).

## La República de Platón

Nuestro modelo de lenguaje va a estar basado en la república de Platón. Este libro está estructurado en forma de una conversación que trata el tema del orden y la justicia dentro de una ciudad. El texto completo está disponible para el dominio público dentro del [proyecto Gutenberg](http://www.gutenberg.org/).

Este libro de Platón está disponible en varios formatos en el [proyecto Gutenberg](http://www.gutenberg.org/cache/epub/1497/pg1497.txt). La versión que nos interesa a nosotros es la versión ASCII del libro. Con la siguiente instrucción puedes descargar el libro donde se han eliminado la portada y la contraportada. 

In [2]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt -O republic.txt

--2021-05-13 15:22:30--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 657826 (642K) [text/plain]
Saving to: ‘republic.txt’

republic.txt        100%[===================>] 642.41K  --.-KB/s    in 0.01s   

2021-05-13 15:22:30 (63.7 MB/s) - ‘republic.txt’ saved [657826/657826]



## Preparación de los datos

Vamos a preparar los datos para construir nuestro modelo. 

### Revisando el texto

Vamos a comenzar revisando parte del texto.

In [3]:
!head -30 republic.txt

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner they would
celebrate the festival, which was a new thing. I was delighted with the
procession of the inhabitants; but that of the Thracians was equally,
if not more, beautiful. When we had finished our prayers and viewed the
spectacle, we turned in the direction of the city; and at that instant
Polemarchus the son of Cephalus chanced to catch sight of us from a
distance as we were starting on our way home, and told his servant to
run and bid us wait for him. The servant took hold of me by the cloak
behind, and said: Polemarchus desires you to wait.

I turned round, and asked him where his master was.

There he is, said the youth, coming after you, if you will only wait.

Certainly we will, said Glaucon; and in a few minutes Polemarchus
appeared, and with him Adeimantus, Glaucon's b

A partir de un rápido vistazo al fragmento de texto anterior podemos ver ciertas cuestiones que tendremos que procesar:
- Las cabeceras de los capítulos.
- Muchos signos de puntuación.
- Nombres extraños.
- Algunos monólogos muy largos. 

### Cargando el texto

El primer paso consiste en cargar el texto en memoria. Podemos desarrollar una pequeña función que se encargue de esto. 

In [4]:
def load_doc(filename):
  # Abrimos el fichero en modo lectura
  file = open(filename,'r')
  # Leemos el texto completo
  text = file.read()
  # Cerramos el fichero
  file.close()
  return text

Usando dicha función podemos cargar nuestro fichero del siguiente modo.

In [5]:
in_filename = 'republic.txt'
doc = load_doc(in_filename)

Ahora podemos mostrar parte de dicho texto.

In [6]:
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


### Limpiando el texto

Ahora necesitamos transformar el texto en bruto a una secuencia de tokens (o palabras) que podamos usar para entrenar nuestro modelo. 

Vamos a aplicar las siguientes operaciones para limpiar nuestro texto:
- Reemplazar todas las ocurrencias de '-' con un espacio en blanco de manera que podamos partir mejor las palabras.
- Partir las palabras basándonos en espacios en blanco.
- Eliminar todos los símbolos de puntuación.
- Eliminar todas las palabras que no son alfabéticas. 
- Normalizar todas las palabras a minúsculas.

La mayoría de estas transformaciones tienen como objetivo reducir el tamaño del vocabulario. Un tamaño de vocabulario excesivamente grande es un problema cuando se intenta crear modelos de lenguaje. Vocabularios pequeños producen modelos más pequeños que se entrenan más rápidos.

Vamos a implementar todas las operaciones de limpieza en la siguiente función.

In [7]:
import string
import re

def clean_doc(doc):
  # Reemplazar '--' con un espacio en blanco ' '
  doc = doc.replace('--',' ')
  # Partir palabras basándonos en espacios en blanco
  tokens = doc.split()
  # Vamos a escapar las palabras para poder filtrarlas por caracteres
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # Eliminamos los símbolos de puntuación
  tokens = [re_punc.sub('',w) for w in tokens]
  # Eliminamos elementos que nos son alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  # Convertimos a minúsculas
  tokens = [word.lower() for word in tokens]
  return tokens
  

Procedemos a limpiar nuestro documento y a continuación mostramos algunas estadísticas sobre nuestro vocabulario.

In [8]:
tokens = clean_doc(doc)
print(tokens[:200])

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [9]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

Total Tokens: 118684
Unique Tokens: 7409


Es decir, nuestro modelo consta de una 7500 palabras. Este tamaño de vocabulario es pequeño y va a ser manejable.

### Guardando el texto limpio

Vamos a organizar la larga lista de tokens en secuencias de 50 palabras de entrada y 1 palabra de salida (esto servirá para luego entrenar nuestro modelo). 

Este proceso lo implementamos con la siguiente función.

In [10]:
def organize_tokens(tokens,input_len=50,output_len=1):
  length = input_len + output_len
  sequences = list()
  for i in range(length,len(tokens)):
    # Elegimos la secuencia de tokens
    seq = tokens[i-length:i]
    # Convertimos la secuencia en una línea
    line = ' '.join(seq)
    # Almacenamos el resultado
    sequences.append(line)
  return sequences

Organizamos nuestros tokens. 

In [11]:
lines = organize_tokens(tokens)

Ahora vamos a guardar las secuencias en un nuevo fichero para poder cargarlo en el futuro. Para ello nos definimos la siguiente función que guardará cada elemento de la secuencia en una línea del fichero. 

In [12]:
def save_doc(lines,filename):
  data = '\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close()

Podemos llamar a la función anterior para guardar nuestro fichero.

In [13]:
out_filename = 'republic_sequences.txt'
save_doc(lines,out_filename)

Podemos ver parte de dicho fichero.

In [14]:
!head -5 republic_sequences.txt

book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was
i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted
i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with
went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would

## Entrenando el modelo de lenguaje

Vamosa  entrenar ahora nuestro modelo a partir de los datos que hemos preparado. Dicho modelo tendrá ciertas características:
- Usará una representación para las palabras de manera que palabras diferentes con significados similares tendrán una representación similar.
- La representación será aprendida al mismo tiempo que se aprende el modelo.
- Aprenderá a predecir la probabilidad de la siguiente palabra a partir del contexto de las últimas 100 palabras.

En concreto para implementar este modelo vamos a usar una capa de Embedding para aprender la representación de las palabras, y una red neuronal recurrente con capas LSTM para predecir nuevas palabras basándonos en el contexto. 

### Cargando las secuencias

Podemos comenzar cargando las secuencias que hemos guardado anteriormente. En este caso este paso no sería necesario ya que el proceso de generación de las secuencias es bastante rápido, pero si estamos trabajando con un dataset más grande sí que puede ser conveniente. 


In [15]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

### Codificando las secuencias

Las capas de Embedding esperan que las secuencias de entrada estén compuestas de vectores de enteros. Para ello vamos a identificar cada palabra de nuestro vocabulario con un entero único y codificarlo en una secuencia de entrada. En el futuro cuando vayamos a realizar las predicciones tendremos que realizar el proceso inverso.

Para llevar a cabo este proceso de tokenización vamos a usar la API de Keras del siguiente modo. 

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

Ahora podemos acceder a los identificadores de cada palabra usando el atributo ``word_index`` del objeto ``Tokenizer`` que hemos creado. 

Además debemos determinar el tamaño de nuestro vocabulario para definir la capa de embedding. En concreto, a las palabras de nuestro vocabulario se les han asignado valores entre 1 y el número total de palabras de nuestro vocabulario. 

In [18]:
vocab_size = len(tokenizer.word_index) + 1 

### Secuencias de entrada y salida

Una vez que tenemos codificadas nuestras secuencias tenemos que separarlas en elementos de entrada ($X$) y de salida ($y$). Después de realizar la separación debemos codificar cada palabra usando el método one-hot. Este proceso lo llevaremos a cabo mediante la función ``to_categorical()`` de Keras.
Finalmente necesitamos especificar cómo de largas serán las secuencias de entrada. 

In [19]:
from tensorflow.keras.utils import to_categorical
from numpy import array 

sequences = array(sequences)
X,y=sequences[:,:-1], sequences[:,-1]
y = to_categorical(y,num_classes=vocab_size)
seq_length = X.shape[1]

### Entrenando el modelo

Ahora podemos definir nuestro modelo que constará de una capa de Embedding, seguida de dos capas LSTM y terminando con una red completamente conectada. 

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

def define_model(vocab_size,seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size,50,input_length=seq_length))
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100,activation='relu'))
  model.add(Dense(vocab_size,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
  return model

Pasamos a entrenar nuestro modelo. Como este proceso es bastante costoso (incluso usando GPUs) en la siguiente sección se proporcionan los ficheros necesarios para usar el modelo. 

In [20]:
model = define_model(vocab_size,seq_length)
model.fit(X,y,batch_size=128,epochs=100)

Epoch 1/100


KeyboardInterrupt: ignored

Una vez entrenado podemos guardar los pesos del modelo y el tokenizador. 

In [21]:
model.save_weights('./model.h5', overwrite=True)

NameError: ignored

In [1]:
from pickle import dump
dump(tokenizer, open('tokenizer.pkl','wb'))

NameError: ignored

## Usando el modelo

Como has podido ver en el paso anterior, el proceso de entrenar este tipo de modelos es muy costoso, por lo que puedes descargar los ficheros necesarios para usar el modelo desde el siguiente enlace. 

In [23]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5 -O model.h5
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl -O tokenizer.pkl

--2021-05-13 15:23:20--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101248 (4.9M) [application/octet-stream]
Saving to: ‘model.h5’

model.h5            100%[===================>]   4.86M  --.-KB/s    in 0.07s   

2021-05-13 15:23:21 (74.5 MB/s) - ‘model.h5’ saved [5101248/5101248]

--2021-05-13 15:23:21--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353379 (345K)

### Cargando los datos

Comenzamos cargando nuestros datos al igual que antes. 

In [24]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Necesitamos este texto para elegir una secuencia de inicio que será la entrada para nuestro modelo. 

In [25]:
seq_length = len(lines[0].split())-1

### Cargando el modelo

Vamos a cargar el modelo y a fijar los pesos. Notar que para este paso ya no necesitamos el uso de TPU, y que el modelo podría ser usado en cualquier ordenador.

In [26]:
model = define_model(vocab_size,seq_length)
model.load_weights('./model.h5')

También necesitamos cargar el tokenizador.

In [27]:
from pickle import load
tokenizer = load(open('tokenizer.pkl','rb'))

### Generando texto

El primer paso para generar el texto consiste en preparar una entrada, para lo cual elegiremos una línea aleatoria del texto. 

In [28]:
from random import randint
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

a rough and ready cure an emetic or a purge or a cautery or the knife these are his remedies and if some one prescribes for him a course of dietetics and tells him that he must swathe and swaddle his head and all that sort of thing he replies at



A continuación podemos generar nuevas palabras una por una. Primero, el texto debe codificarse usando el tokenizer que hemos cargado anteriormente. Ahora el modelo puede predecir nuevas palabras usando el método ``predict_classes()`` que devuelve el índice de la palabra con probabilidad más alta. 

Esta palabra se añade a nuestro texto inicial y se repite el proceso. Notar que esta secuencia va a ir creciendo por lo que tendremos que truncarla, para lo que utilizamos la función ``pad_sequences()`` de Keras. Todo este proceso se puede implementar con la siguiente función. 

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
  result = list()
  in_text = seed_text
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    yhat = model.predict_classes(encoded,verbose=0)
    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

Ahora podemos generar una nueva secuencia usando el siguiente código. Cada vez que lo ejecutemos obtendremos un resultado distinto.

In [30]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model,tokenizer,seq_length,seed_text,50)
print(generated)

soul on which they mightily fasten imparting grace and making the soul of him who is rightly educated graceful or of him who is illeducated ungraceful and also because he who has received this true education of the inner being will most shrewdly perceive omissions or faults in art and nature



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


and the other excellences of peirithous taking who have won a man who is wealthy such magic degraded and do you mean that the other features their mode of speaking and the other principle is arrayed on the state and the other timid the followers and the other plunged in


## Ejercicio

Elige tu propio libro del proyecto Gutenberg (es posible usar libros en [español](https://www.gutenberg.org/browse/languages/es)) y crea tu propio modelo de lenguaje. 

Descargamos el libro.

In [31]:
!wget https://www.gutenberg.org/cache/epub/1/pg1.txt.utf8 -O texto.txt

--2021-05-13 15:30:57--  https://www.gutenberg.org/cache/epub/1/pg1.txt.utf8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120946 (118K) [text/plain]
Saving to: ‘texto.txt’

texto.txt           100%[===================>] 118.11K  --.-KB/s    in 0.03s   

2021-05-13 15:30:58 (3.48 MB/s) - ‘texto.txt’ saved [120946/120946]



Observamos el principio del texto.

In [32]:
!head -30 texto.txt

﻿


     NOTE:  This file combines the first two Project Gutenberg
     files, both of which were given the filenumber #1. There are
     several duplicate files here. There were many updates over
     the years.  All of the original files are included in the
     "old" subdirectory which may be accessed under the "More
     Files" listing in the PG Catalog of this file. No changes
     have been made in these original etexts.



**Welcome To The World of Free Plain Vanilla Electronic Texts**

**Etexts Readable By Both Humans and By Computers, Since 1971**

*These Etexts Prepared By Hundreds of Volunteers and Donations*

Below you will find the first nine Project Gutenberg Etexts, in
one file, with one header for the entire file.  This is to keep
the overhead down, and in response to requests from Gopher site
keeper to eliminate as much of the headers as possible.

However, for legal and financial reasons, we must request these
headers be left at the beginning of each file that is post

Cargamos el texto en memoria y mostramos parte del fichero.

In [33]:
def load_doc(filename):
  # Abrimos el fichero en modo lectura
  file = open(filename,'r')
  # Leemos el texto completo
  text = file.read()
  # Cerramos el fichero
  file.close()
  return text

in_filename = 'texto.txt'
doc = load_doc(in_filename)

print(doc[:200])

﻿


     NOTE:  This file combines the first two Project Gutenberg
     files, both of which were given the filenumber #1. There are
     se


Limpiamos el documento y mostramos algunas estadísticas sobre el texto.

In [35]:
import string
import re

def clean_doc(doc):
  # Reemplazar '--' con un espacio en blanco ' '
  doc = doc.replace('--',' ')
  # Partir palabras basándonos en espacios en blanco
  tokens = doc.split()
  # Vamos a escapar las palabras para poder filtrarlas por caracteres
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # Eliminamos los símbolos de puntuación
  tokens = [re_punc.sub('',w) for w in tokens]
  # Eliminamos elementos que nos son alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  # Convertimos a minúsculas
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_doc(doc)
print(tokens[:200])

print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

['note', 'this', 'file', 'combines', 'the', 'first', 'two', 'project', 'gutenberg', 'files', 'both', 'of', 'which', 'were', 'given', 'the', 'filenumber', 'there', 'are', 'several', 'duplicate', 'files', 'here', 'there', 'were', 'many', 'updates', 'over', 'the', 'years', 'all', 'of', 'the', 'original', 'files', 'are', 'included', 'in', 'the', 'old', 'subdirectory', 'which', 'may', 'be', 'accessed', 'under', 'the', 'more', 'files', 'listing', 'in', 'the', 'pg', 'catalog', 'of', 'this', 'file', 'no', 'changes', 'have', 'been', 'made', 'in', 'these', 'original', 'etexts', 'welcome', 'to', 'the', 'world', 'of', 'free', 'plain', 'vanilla', 'electronic', 'texts', 'etexts', 'readable', 'by', 'both', 'humans', 'and', 'by', 'computers', 'since', 'these', 'etexts', 'prepared', 'by', 'hundreds', 'of', 'volunteers', 'and', 'donations', 'below', 'you', 'will', 'find', 'the', 'first', 'nine', 'project', 'gutenberg', 'etexts', 'in', 'one', 'file', 'with', 'one', 'header', 'for', 'the', 'entire', 'file

Guardamos el texto limpio y mostramos cómo quedaría.

In [37]:
def organize_tokens(tokens,input_len=50,output_len=1):
  length = input_len + output_len
  sequences = list()
  for i in range(length,len(tokens)):
    # Elegimos la secuencia de tokens
    seq = tokens[i-length:i]
    # Convertimos la secuencia en una línea
    line = ' '.join(seq)
    # Almacenamos el resultado
    sequences.append(line)
  return sequences

lines = organize_tokens(tokens)

def save_doc(lines,filename):
  data = '\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close()

out_filename = 'texto_limpio.txt'
save_doc(lines,out_filename)

!head -5 texto_limpio.txt

note this file combines the first two project gutenberg files both of which were given the filenumber there are several duplicate files here there were many updates over the years all of the original files are included in the old subdirectory which may be accessed under the more files listing in
this file combines the first two project gutenberg files both of which were given the filenumber there are several duplicate files here there were many updates over the years all of the original files are included in the old subdirectory which may be accessed under the more files listing in the
file combines the first two project gutenberg files both of which were given the filenumber there are several duplicate files here there were many updates over the years all of the original files are included in the old subdirectory which may be accessed under the more files listing in the pg
combines the first two project gutenberg files both of which were given the filenumber there are several duplicat

Cargamos y codificamos las secuencias.

In [39]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

in_filename = 'texto_limpio.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

vocab_size = len(tokenizer.word_index) + 1 

Separamos secuencias en entrada y salida y entrenamos el modelo.

In [41]:
from tensorflow.keras.utils import to_categorical
from numpy import array 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

sequences = array(sequences)
X,y=sequences[:,:-1], sequences[:,-1]
y = to_categorical(y,num_classes=vocab_size)
seq_length = X.shape[1]

def define_model(vocab_size,seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size,50,input_length=seq_length))
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100,activation='relu'))
  model.add(Dense(vocab_size,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
  return model

model = define_model(vocab_size,seq_length)
model.fit(X,y,batch_size=128,epochs=100)

Epoch 1/100
153/153 [==============================] - 6s 22ms/step - loss: 7.0969 - accuracy: 0.0559
Epoch 2/100
153/153 [==============================] - 3s 20ms/step - loss: 6.2558 - accuracy: 0.0706
Epoch 3/100
153/153 [==============================] - 5s 30ms/step - loss: 6.1505 - accuracy: 0.0669
Epoch 4/100
153/153 [==============================] - 4s 27ms/step - loss: 5.9701 - accuracy: 0.0904
Epoch 5/100
153/153 [==============================] - 3s 21ms/step - loss: 5.7854 - accuracy: 0.0982
Epoch 6/100
153/153 [==============================] - 3s 22ms/step - loss: 5.6859 - accuracy: 0.1013
Epoch 7/100
153/153 [==============================] - 3s 20ms/step - loss: 5.6056 - accuracy: 0.1093
Epoch 8/100
153/153 [==============================] - 3s 21ms/step - loss: 5.4933 - accuracy: 0.1141
Epoch 9/100
153/153 [==============================] - 3s 21ms/step - loss: 5.3856 - accuracy: 0.1188
Epoch 10/100
153/153 [==============================] - 4s 25ms/step - loss: 5.304

Guardamos los pesos del modelo y el Tokenizador.

In [42]:
from pickle import dump
model.save_weights('./model.h5', overwrite=True)

dump(tokenizer, open('tokenizer.pkl','wb'))

Cargamos los datos y el modelo.

In [43]:
from pickle import load

in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

seq_length = len(lines[0].split())-1

model = define_model(vocab_size,seq_length)
model.load_weights('./model.h5')

tokenizer = load(open('tokenizer.pkl','rb'))

Generamos el texto.

In [44]:
from random import randint
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
  result = list()
  in_text = seed_text
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    yhat = model.predict_classes(encoded,verbose=0)
    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

rightly answer that question whichever of the two are best able to guard the laws and institutions of our state let them be our guardians very good neither i said can there be any question that the guardian who is to keep anything should have eyes rather than no eyes there



Generamos una nueva secuencia.

In [45]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model,tokenizer,seq_length,seed_text,50)
print(generated)

progress certainly not and again if he is forgetful and retains nothing of what he learns will he not be an empty vessel that is certain labouring in vain he must end in hating himself and his fruitless occupation yes then a soul which forgets cannot be ranked among genuine philosophic



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


the accused shall enjoy virtue shall be used to be a witness and if we how it must needs be are attempted we need these just god about great donations donations if you wish to be pursued revived without man would independent among himself for every country a grand and


Recuerda guardar este notebook en tu repositorio usando la opción "Save in GitHub" del menú File.